# Data preparation and analysis for chat model fine-tuning

This notebook serves as a tool to preprocess and analyze the chat dataset used for fine-tuning a chat model.
It checks for format errors, provides basic statistics, and estimates token counts for fine-tuning costs.
The method shown here corresponds to the [current fine-tuning method](https://platform.openai.com/docs/guides/fine-tuning) for gpt-3.5-turbo.
See [legacy fine-tuning](https://platform.openai.com/docs/guides/legacy-fine-tuning) for models like babbage-002 and davinci-002.

In [ ]:
# prompt: pip install tiktoken

!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00


In [ ]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

## Data loading

We first load the chat dataset from an [example JSONL file](https://github.com/openai/openai-cookbook/blob/main/examples/data/toy_chat_fine_tuning.jsonl).

In [ ]:
data_path = "drake.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))

print("First example:")
for message in dataset[0]["messages"]:
    print(message)

print('\n\n\n')

print("10th example:")
for message in dataset[9]["messages"]:
    print(message)

Num examples: 725
First example:
{'role': 'system', 'content': 'You are a songwriter.'}
{'role': 'user', 'content': 'I want to write a Drake song.'}
{'role': 'assistant', 'content': 'Ok, tell me a verse of the song using some phrases.'}
{'role': 'user', 'content': 'phones off, no calls, hit the stride'}
{'role': 'assistant', 'content': "I been in the crib with the phones off\nI been at the house takin' no calls\nI done hit the stride, got my shit going\nIn the 6 cookin' with the wri-wri-wri-wri"}




10th example:
{'role': 'system', 'content': 'You are a songwriter.'}
{'role': 'user', 'content': 'I want to write a Drake song.'}
{'role': 'assistant', 'content': 'Ok, tell me a verse of the song using some phrases.'}
{'role': 'user', 'content': 'Trust me when I say I thought it all through,Tried to kill me but they still want the Views,I am very much alive on the six side'}
{'role': 'assistant', 'content': 'Make me feel the same way that I did to you\nTrust me when I say I thought it all 

## Format validation

We can perform a variety of error checks to validate that each conversation in the dataset adheres to the format expected by the fine-tuning API. Errors are categorized based on their nature for easier debugging.

1. **Data Type Check**: Checks whether each entry in the dataset is a dictionary (`dict`). Error type: `data_type`.
2. **Presence of Message List**: Checks if a `messages` list is present in each entry. Error type: `missing_messages_list`.
3. **Message Keys Check**: Validates that each message in the `messages` list contains the keys `role` and `content`. Error type: `message_missing_key`.
4. **Unrecognized Keys in Messages**: Logs if a message has keys other than `role`, `content`, `weight`, `function_call`, and `name`. Error type: `message_unrecognized_key`.
5. **Role Validation**: Ensures the `role` is one of "system", "user", or "assistant". Error type: `unrecognized_role`.
6. **Content Validation**: Verifies that `content` has textual data and is a string. Error type: `missing_content`.
7. **Assistant Message Presence**: Checks that each conversation has at least one message from the assistant. Error type: `example_missing_assistant_message`.

The code below performs these checks, and outputs counts for each type of error found are printed. This is useful for debugging and ensuring the dataset is ready for the next steps.


# Remove any messages with errors and make 3 new cleaned files, for train, test, validation.

In [ ]:
import random

def generate_builder_chat_file_cleaned(train, test, val, examples):
    """
    Each line is {messages:[{}, {}, {}]}. Each line is about a song from a particular
    artist, chatted to completion of the song.

    Args:
    artist: string: if artist is not None, then only generate messages about a particular
    artist.
    """
    with open(train, 'w') as train, open(test, 'w') as test, open(val, 'w') as val:
      for example in examples:
        random_number = random.randint(0, 99)
        if random_number < 70:
          chat_data = example
          json_line = json.dumps(chat_data)
          train.write(json_line + "\n")
        elif random_number < 90:
          chat_data = example
          json_line = json.dumps(chat_data)
          test.write(json_line + "\n")
        else:
          chat_data = example
          json_line = json.dumps(chat_data)
          val.write(json_line + "\n")

In [ ]:
# Format error checks
format_errors = defaultdict(int)

goodegs = []

for ex in dataset:
    good = True
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        good = False
        continue

    messages = ex.get("messages", None)
    if not messages:
        good = False
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            good = False
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            good = False
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            good = False
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            good = False
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        good = False
        format_errors["example_missing_assistant_message"] += 1

    if good:
        goodegs.append(ex)

if format_errors:
    print("Found errors before cleaning:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
    generate_builder_chat_file_cleaned("train.jsonl", "test.jsonl", "val.jsonl", goodegs)
else:
    print("No errors found")

Found errors before cleaning:
missing_content: 18


# Cleaned data validation

In [ ]:
data_path = "train.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
NUM_EXAMPLES_FULL = len(dataset)
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 486
First example:
{'role': 'system', 'content': 'You are a songwriter.'}
{'role': 'user', 'content': 'I want to write a Drake song.'}
{'role': 'assistant', 'content': 'Ok, tell me a verse of the song using some phrases.'}
{'role': 'user', 'content': 'church on Sunday, not in Kansas anymore, Oh Lord'}
{'role': 'assistant', 'content': "Your momma used to live at the church on Sunday\nYou just go to LIV after church on Sunday\nOh Lord, oh Lord we're not in Kansas anymore\nWe're not in Kansas anymore"}


In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


# Subsample the train, test, val split

In [ ]:
!pip install dask

In [ ]:
import dask.bag as db
import json

samples = {
    'SMALLEST': 0.01,
    'SMALLER': 0.05,
    'SMALL': 0.1,
    'MEDIUM_SMALL': 0.2,
    'MEDIUM_MEDIUM': 0.3,
    'MEDIUM_LARGE': 0.4,
    'LARGE': 0.5,
    'LARGER': 0.7,
    'EVEN_LARGER': 0.85
}

lines = db.read_text('train.jsonl')
data = lines.map(json.loads)

print(f'full sized dataset is {NUM_EXAMPLES_FULL} lines')
for name, size in samples.items():
  sampled_data = data.random_sample(size)

  result = sampled_data.compute()

  with open(f'train_{size}.jsonl', 'w') as file:
    print(f'writing a training file with {len(result)} lines')
    for item in result:
      json.dump(item, file)
      file.write('\n')

full sized dataset is 486 lines
writing a training file with 1 lines
writing a training file with 17 lines
writing a training file with 61 lines
writing a training file with 100 lines
writing a training file with 143 lines
writing a training file with 222 lines
writing a training file with 233 lines
writing a training file with 353 lines
writing a training file with 413 lines


# Validating subsamples

In [ ]:
# TODO

## Token Counting Utilities

Lets define a few helpful utilities to be used in the rest of the notebook.

In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## Data Warnings and Token Counts

With some lightweight analysis we can identify potential issues in the dataset, like missing messages, and provide statistical insights into message and token counts.

1. **Missing System/User Messages**: Counts the number of conversations missing a "system" or "user" message. Such messages are critical for defining the assistant's behavior and initiating the conversation.
2. **Number of Messages Per Example**: Summarizes the distribution of the number of messages in each conversation, providing insight into dialogue complexity.
3. **Total Tokens Per Example**: Calculates and summarizes the distribution of the total number of tokens in each conversation. Important for understanding fine-tuning costs.
4. **Tokens in Assistant's Messages**: Calculates the number of tokens in the assistant's messages per conversation and summarizes this distribution. Useful for understanding the assistant's verbosity.
5. **Token Limit Warnings**: Checks if any examples exceed the maximum token limit (4096 tokens), as such examples will be truncated during fine-tuning, potentially resulting in data loss.


In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 5, 5
mean / median: 5.0, 5.0
p5 / p95: 5.0, 5.0

#### Distribution of num_total_tokens_per_example:
min / max: 62, 940
mean / median: 152.31034482758622, 138.0
p5 / p95: 85.0, 224.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 25, 877
mean / median: 100.42468239564428, 83.0
p5 / p95: 37.0, 163.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


## Cost Estimation

In this final section, we estimate the total number of tokens that will be used for fine-tuning, which allows us to approximate the cost. It is worth noting that the duration of the fine-tuning jobs will also increase with the token count.

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~4306 tokens that will be charged for during training
By default, you'll train for 20 epochs on this dataset
By default, you'll be charged for ~86120 tokens


10$ per million tokens with GPT-4.

See https://openai.com/pricing to estimate total costs.